# BibRec

## Exploration of Normalized BookCrossing Dataset

In [1]:
from bibrec.server.Utils import *
import seaborn as sns

FileNotFoundError: [Errno 2] No such file or directory: '../../data/BX-Books.csv'

In [ ]:
norm_books = get_books(books_path='data/normalized_books.csv')
norm_users = get_users(users_path='data/normalized_users.csv')
norm_ratings = get_ratings(ratings_path='data/normalized_ratings.csv')

In [ ]:
norm_books, norm_users, norm_ratings = normalize_data(norm_books, norm_users, norm_ratings)

In [ ]:
norm_books, norm_users, norm_ratings = get_normalized_data(books_path='data/normalized_books.csv',
                                                           users_path='data/normalized_users.csv',
                                                           ratings_path='data/normalized_ratings.csv')

### Books

In [ ]:
norm_books

In [ ]:
norm_books.info()

In [ ]:
norm_books.describe()

In [ ]:
sns.heatmap(norm_books.isnull(), cbar = False, cmap='coolwarm')

### Users

In [ ]:
norm_users

In [ ]:
norm_users.info()

In [ ]:
norm_users.describe()

In [ ]:
sns.heatmap(norm_users.isnull(), cbar = False, cmap='coolwarm')

In [ ]:
# find out top countries
get_top_countries(norm_users)

In [ ]:
# find out top states
top_states = norm_users.state.value_counts()[:top_n].index.tolist()
top_states = list(map(lambda x: str(x).strip().lower().replace(' ', '_'), top_states))
if 'other' not in top_states:
    top_states.append("other")
top_states

### Ratings

In [ ]:
norm_ratings

In [ ]:
norm_ratings.info()

In [ ]:
norm_ratings.describe()

In [ ]:
import seaborn as sns
sns.countplot(x=norm_ratings["book_rating"])

In [ ]:
explicit_ratings = get_explicit_ratings(norm_ratings)
sns.countplot(x=explicit_ratings["book_rating"])

## Hot Encoded Dataset

In [ ]:
from bibrec.server.Utils import hot_encode_books, hot_encode_users
encoded_books = hot_encode_books(norm_books)
encoded_books.to_csv('data/encoded_books.csv')

In [ ]:
encoded_users = hot_encode_users(norm_users)
encoded_users.to_csv('data/encoded_users.csv')

In [ ]:
encoded_books.describe()

In [ ]:
encoded_users.describe()